In [ ]:
import pandas as pd
import requests
from playwright.async_api import async_playwright
import zlib

# from playwright.sync_api import sync_playwright


In [ ]:
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless=False)
page = await browser.new_page()


In [ ]:
def generate_hash_filename(url):
    """Generate a hash of the URL to use as a filename."""
    url_hash = zlib.crc32(url.encode())
    file_name = f"{url_hash:x}"
    return file_name


In [ ]:
save_path = "../../data/data_30_10"
url = "https://help.byupathway.edu/knowledgebase/"


In [ ]:
await page.goto(url)
await page.wait_for_load_state()


In [ ]:
while True:
    try:
        await page.click("text=Show More...")
        button = page.query_selector("#pagingButton")
    except:
        break


/tmp/ipykernel_16539/1682996073.py:4: RuntimeWarning: coroutine 'Page.query_selector' was never awaited
  button = page.query_selector("#pagingButton")


In [ ]:
# get the element with id articleList
articles = await page.query_selector("#articleList")
# get all the links inside it, the link is the a tag, the title in the h5 tag inside the a tag and the descripcion in the p tag inside the a tag
links = await articles.query_selector_all("a")
data = []
for link in links:
    title = await link.query_selector("h5")
    description = await link.query_selector("p")
    filename = generate_hash_filename(url + await link.get_attribute("href"))
    data.append(
        {
            "title": await title.inner_text(),
            "description": await description.inner_text(),
            "url": url + await link.get_attribute("href"),
            "filename": filename,
        }
    )

df = pd.DataFrame(data)
df


,title,description,url,filename
0,How long does it take to get a decision,How long will it take to get my admission deci...,https://help.byupathway.edu/knowledgebase/arti...,200631eb
1,Academic Misconduct Policies for PathwayConnec...,What are the policies for academic misconduct ...,https://help.byupathway.edu/knowledgebase/arti...,69b665df
2,PC - Disruptive Behavior,.,https://help.byupathway.edu/knowledgebase/arti...,c08db428
3,Institute/Religion Course Information,What are the requirements for transferring Ins...,https://help.byupathway.edu/knowledgebase/arti...,e49f35bd
4,Autodrop for EnglishConnect,What is Autodrop for EnglishConnect?,https://help.byupathway.edu/knowledgebase/arti...,e55d5f8a
...,...,...,...,...
122,Admission Requirements,Is there anything that could stop me from join...,https://help.byupathway.edu/knowledgebase/arti...,2b532722
123,Submitting Transfer Credits,Where do I go to submit my transfer credits?,https://help.byupathway.edu/knowledgebase/arti...,c49f2c39
124,Inviting Friends to Pathway,What are some ways I can encourage someone I k...,https://help.byupathway.edu/knowledgebase/arti...,ba2e5084
125,Viewing Account Holds,How do I view the holds on my account?,https://help.byupathway.edu/knowledgebase/arti...,34e7b84d


In [11]:
# save the df in a csv file
df.to_csv(f"{save_path}/articles.csv", index=False)


In [ ]:
for index, row in df.iterrows():
    await page.goto(row["url"])
    await page.wait_for_load_state()
    content = await page.query_selector(".wrapper-body")
    post_content = await content.inner_html()
    if post_content:
        with open(f"{save_path}/crawl/{row["filename"]}.html", "w") as f:
            f.write(post_content)
    

In [ ]:
await browser.close()
